In [2]:
import ctypes
from tinygrad import Tensor 

In [41]:
metal = ctypes.CDLL(ctypes.util.find_library("metal"))
print(ctypes.util.find_library("Metal"))

/System/Library/Frameworks/Metal.framework/Metal


In [36]:
metal.MTLCreateSystemDefaultDevice.restype = c_
metal.MTLCreateSystemDefaultDevice()


0

In [33]:
# metal = ctypes.CDLL("/System/Library/Frameworks/Metal.framework/Versions/A/Resources/BridgeSupport/Metal.dylib")

In [99]:
device = metal.MTLCreateSystemDefaultDevice()
device
# ctypes assumes int as the default return type. To set a custom return type we need to create python classes from the header files of metal. unfortunately, it's not that straightfoward as clang2py only supports c headers and apple gives us c++ headers

0

In [6]:
a = Tensor.ones(5)
b = Tensor.ones(5)
a

<Tensor <LB GPU (5,) contig:False ShapeTracker(views=(View(shape=(5,), strides=(0,), offset=0, mask=None, contiguous=False),))> on GPU with grad None>

In [7]:
print(device.supportsFamily(1009))

AttributeError: 'int' object has no attribute 'supportsFamily'

In [10]:
import llvmlite.binding as llvm
from llvmlite import ir
from ctypes import CFUNCTYPE, c_double

In [91]:
double = ir.DoubleType()
fnty = ir.FunctionType(double, (double, double))
module = ir.Module(name="mod")
func = ir.Function(module, fnty, name="fpadd")

block = func.append_basic_block(name="entry")
builder = ir.IRBuilder(block)
a, b = func.args
result = builder.fadd(a, b, name="res")
builder.ret(result)

llvm_ir = str(module)
print(llvm_ir)

; ModuleID = "mod"
target triple = "unknown-unknown-unknown"
target datalayout = ""

define double @"fpadd"(double %".1", double %".2")
{
entry:
  %"res" = fadd double %".1", %".2"
  ret double %"res"
}



In [71]:
llvm.initialize()
llvm.initialize_native_target()
llvm.initialize_native_asmprinter()
target = llvm.Target.from_default_triple()
target_machine = target.create_target_machine()
backing_mod = llvm.parse_assembly("")
engine = llvm.create_mcjit_compiler(backing_mod, target_machine)

In [88]:
mod = llvm.parse_assembly(llvm_ir)
mod.verify()
engine.add_module(mod)
engine.finalize_object()
engine.run_static_constructors()

In [95]:
func_ptr = engine.get_function_address("fpadd")
cfunc = CFUNCTYPE(c_double, c_double, c_double)(func_ptr)
res = cfunc(1.0, 3.5)
print("fpadd(...) =", res)

fpadd(...) = 4.5


In [98]:
print(target_machine.emit_assembly(mod))

	.section	__TEXT,__text,regular,pure_instructions
	.build_version macos, 14, 0
	.globl	_fpadd
	.p2align	2
_fpadd:
	.cfi_startproc
	fadd	d0, d0, d1
	ret
	.cfi_endproc

.subsections_via_symbols



In [1]:
# llvm apparently doesn't allow adding custom backends (targets) out of source. it's not straightforward to add apple amx to llvm then...

In [14]:
import llvmlite.utils as ll
ll.get_library_files()
llvm.initialize_all_targets()
ll